In [1]:
!pip install gdown

In [2]:
!pip install transformers datasets torch gdown optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 17.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64

In [3]:
!pip install hazm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 23.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 29.6 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9296 sha256=16f2bc5e0aa6d1bab27469f712d8238f4cd89c2a4d92ae279093a32fa1b1b9e5
  Stored in directory: /root/.cache/pip/wheels/bc/be/39/c37ad168eb2ff644c9685f52554440372129450f0b8ed203dd
Successfully built flashtext
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This

In [1]:
import gdown

# Google Drive file ID
file_id = '1YgrCYY-Z0h2z0-PfWVfOGt1Tv0JDI-qz'
# Construct the URL to download the file
url = f'https://drive.google.com/uc?id={file_id}'
output = 'dataset.zip'

# Download the file
gdown.download(url, output, quiet=False)

# Extract the dataset
import zipfile
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall('dataset')


Downloading...
From: https://drive.google.com/uc?id=1YgrCYY-Z0h2z0-PfWVfOGt1Tv0JDI-qz
To: /content/dataset.zip
100%|██████████| 13.6M/13.6M [00:00<00:00, 68.8MB/s]


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer
import re
from hazm import Normalizer
import pandas as pd
from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import BertConfig, BertTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
# Import required packages

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import hazm
import plotly.express as px
import plotly.graph_objects as go
from tqdm.notebook import tqdm
import os
import re
import json
import copy
import collections

In [3]:
normalizer_instance = Normalizer()

# Function to perform additional pre-processing steps
def preprocess_text(input_text):
    # Normalize the text using Hazm normalizer
    input_text = normalizer_instance.normalize(input_text)

    # Remove extra punctuation
    input_text = re.sub(r'[^\w\s]', ' ', input_text)  # Remove punctuation and replace with space

    # Transform Arabic characters to Persian ones
    arabic_to_persian_map = {
        'ي': 'ی', 'ك': 'ک', 'ؤ': 'و', 'ى': 'ی', 'ة': 'ه', 'ۀ': 'ه'
    }
    for arabic_char, persian_char in arabic_to_persian_map.items():
        input_text = input_text.replace(arabic_char, persian_char)

    # Remove repeated letters more than twice in non-standard Persian words
    input_text = re.sub(r'(.)\1{2,}', r'\1\1', input_text)

    # Remove Arabic diacritics
    input_text = re.sub(r'[\u064B-\u065F\u0670]', '', input_text)

    # Replace non-Persian characters with space
    input_text = re.sub(r'[^آ-ی\s]', ' ', input_text)

    # Remove hashtag sign while preserving hashtag information
    input_text = re.sub(r'#(\w+)', r'\1', input_text)

    # Replace Persian numeric characters with English numeric characters
    persian_to_english_digit_map = {
        '۰': '0', '۱': '1', '۲': '2', '۳': '3', '۴': '4',
        '۵': '5', '۶': '6', '۷': '7', '۸': '8', '۹': '9'
    }
    for persian_digit, english_digit in persian_to_english_digit_map.items():
        input_text = input_text.replace(persian_digit, english_digit)

    unicode_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)

    input_text = unicode_pattern.sub(r'', input_text)

    input_text = re.sub("#", "", input_text)
    input_text = re.sub("\s+", " ", input_text)

    return input_text


In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('ViraIntelligentDataMining/AriaBERT')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [6]:
delimiter='\t'# Load dataset
train_df = pd.read_csv('/content/train.csv' , delimiter='\t')
dev_df = pd.read_csv('/content/dev.csv' , delimiter='\t')
test_df = pd.read_csv('/content/test.csv', delimiter='\t')

In [7]:
print('data information')
print(train_df.info(), '\n')

print('missing values:')
print(train_df.isnull().sum(), '\n')


data information
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6896 entries, 0 to 6895
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6896 non-null   int64 
 1   content     6896 non-null   object
 2   label       6896 non-null   object
 3   label_id    6896 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 215.6+ KB
None 

missing values stats
Unnamed: 0    0
content       0
label         0
label_id      0
dtype: int64 



In [48]:
label_encoder = LabelEncoder()
train_df['label'] = label_encoder.fit_transform(train_df['label'])
dev_df['label'] = label_encoder.transform(dev_df['label'])
test_df['label'] = label_encoder.transform(test_df['label'])

In [9]:
# Function to apply additional preprocessing
def preprocess_dataframe(df):
    df['text'] = df['content'].apply(preprocess_text)  # Assuming 'content' is the column containing text
    return df[['text', 'label']]  # Assuming 'label' is the column containing labels

# preprocessing each dataframe
train_df = preprocess_dataframe(train_df)
dev_df = preprocess_dataframe(dev_df)
test_df = preprocess_dataframe(test_df)

In [10]:
# count of data samples per label
label_counts = train_df['label'].value_counts()

print("Count of data samples per label:")
print(label_counts)

Count of data samples per label:
label
3    2245
0    1593
5    1350
2    1304
6     206
1     101
4      97
Name: count, dtype: int64


In [12]:
# Define the class mapping
class_mapping = {4: 6, 1: 6}

# Update labels in the dataset
train_df['label'] = train_df['label'].replace(class_mapping)
dev_df['label'] = dev_df['label'].replace(class_mapping)
test_df['label'] = test_df['label'].replace(class_mapping)

# Get count of data samples per updated label
new_label_counts = train_df['label'].value_counts()

# Display count of data samples per updated label
print("Count of data samples per updated label:")
print(new_label_counts)

Count of data samples per updated label:
label
3    2245
0    1593
5    1350
2    1304
6     404
Name: count, dtype: int64


<ipython-input-12-ef08a291c5f1>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['label'] = test_df['label'].replace(class_mapping)


In [13]:
# Convert dataframes to datasets
train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)
test_dataset = Dataset.from_pandas(test_df)

def tokenize(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length')

train_dataset = train_dataset.map(tokenize, batched=True)
dev_dataset = dev_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/6896 [00:00<?, ? examples/s]

Map:   0%|          | 0/767 [00:00<?, ? examples/s]

Map:   0%|          | 0/852 [00:00<?, ? examples/s]

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'device: {device}')

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

device: cuda:0
CUDA is available!  Training on GPU ...


In [15]:
# general config
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16

EPOCHS = 3
EEVERY_EPOCH = 1000
LEARNING_RATE = 2e-5
CLIP = 0.0

MODEL_NAME_OR_PATH = 'ViraIntelligentDataMining/AriaBERT'
OUTPUT_PATH = '/content/bert-fa-base-uncased-sentiment-taaghceh/pytorch_model.bin'

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [16]:
print("Label encoder classes:", label_encoder.classes_)

Label encoder classes: ['بازی ویدیویی' 'راهنمای خرید' 'سلامت و زیبایی' 'علم و تکنولوژی' 'عمومی'
 'هنر و سینما' 'کتاب و ادبیات']


In [17]:

label2id = {
    "عمومی" :0,
    "بقیه" :6,
    "سلامت و زیبایی" : 2,
    "راهنمای خرید" : 3,
    "کتاب و ادبیات" : 5,
}
id2label = {
    0: "عمومی",
    6: "بقیه",
    2: "سلامت و زیبایی",
    3: "راهنمای خرید",
    5: "کتاب و ادبیات",
}
label2id = {label: id for id, label in id2label.items()}
labels = [item for item in label2id.keys()]
labels.reverse()

In [19]:
# setup the tokenizer and configuration
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('ViraIntelligentDataMining/AriaBERT')
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH, **{
        'label2id': label2id,
        'id2label': id2label,
    })

print(config.to_json_string())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


{
  "_name_or_path": "./MyRoBERTaConfig",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "\u0639\u0645\u0648\u0645\u06cc",
    "2": "\u0633\u0644\u0627\u0645\u062a \u0648 \u0632\u06cc\u0628\u0627\u06cc\u06cc",
    "3": "\u0631\u0627\u0647\u0646\u0645\u0627\u06cc \u062e\u0631\u06cc\u062f",
    "5": "\u06a9\u062a\u0627\u0628 \u0648 \u0627\u062f\u0628\u06cc\u0627\u062a",
    "6": "\u0628\u0642\u06cc\u0647"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "\u0628\u0642\u06cc\u0647": 6,
    "\u0631\u0627\u0647\u0646\u0645\u0627\u06cc \u062e\u0631\u06cc\u062f": 3,
    "\u0633\u0644\u0627\u0645\u062a \u0648 \u0632\u06cc\u0628\u0627\u06cc\u06cc": 2,
    "\u0639\u0645\u0648\u0645\u06cc": 0,
    "\u06a9\u062a\u0627\u0628 \u0648 \u0627\u062f\u0628\u06

In [20]:
train_df[:5]

,text,label
0,نمایش تبلیغ در لاک اسکرین تعدادی از گوشی های ه...,3
1,شکست در باکس آفیس پس از بازخوردهای بد طرفداران...,5
2,کلاسیک بینی همه چیز در یک شب اتفاق افتاد فیلم ...,5
3,اپل دوباره سراغ رنده رفته چراکه آپگرید کردن سط...,3
4,بررسی جز به جز بهترین بخش جعبه ابزار بازی سازا...,0


In [21]:
random_index = np.random.randint(0, len(train_df))
sample_comment = train_df.iloc[random_index]['text']
sample_label = train_df.iloc[random_index]['label']

print(f'Sample: \n{sample_comment}\n{sample_label}')

Sample: 
بهترین های سال فیلم به پایان سال نزدیک می شویم سالی که در دنیای سرگرمی اتفاقات زیاد و جالبی را با خودش به همراه آورده بود درست مثل سال گذشته امسال هم بهترین های سال را در بخش های مختلفی انتخاب کرده ایم و فیلم هم یکی از این بخش ها است طرفدار هر سبکی که باشید مطمئنا امسال فیلمی خوب برای دیدن پیدا کرده اید از فیلم های ابرقهرمانی خوبی مثل جنگ داخلی گرفته تا انیمیشن هایی مثل زوتوپیا یا موآنا و حتی فیلم های عملی تخیلی بدرد بخور فیلم های امسال کم و بیش به مذاق همه خوش آمده است طبق نظرسنجی بلند و بالایی که بین تحریریه ی دیجی کالامگ و عده ای از فیلم دوستان دور و برمان داشته ایم از بین فیلم امسال به پنج فیلم برتر سال رسیده ایم تاکید می کنیم که چیزی به اسم بهترین مطلق وجود ندارد و این فیلم ها پس از رای گیری بین بر و بچه های دیجی کالامگ به دست آمده اند حتما در بخش نظرات از بهترین های فیلم هایی که امسال دیده اید برای ما و دیگران بگویید اگر شاهکاری به اسم محافظان کهکشان را کنار بگذاریم در بین فیلم های ابرقهرمانی چند سال اخیر فکر کنیم به سادگی بتوانیم بگوییم که دکتر استرنج یک سر و گردن بالات

In [22]:
tokens = tokenizer.tokenize(sample_comment)
token_ids = tokenizer.convert_tokens_to_ids(tokens)

print(f'  Comment: {sample_comment}')
print(f'   Tokens: {tokenizer.convert_tokens_to_string(tokens)}')
print(f'Token IDs: {token_ids}')

Token indices sequence length is longer than the specified maximum sequence length for this model (734 > 512). Running this sequence through the model will result in indexing errors


  Comment: بهترین های سال فیلم به پایان سال نزدیک می شویم سالی که در دنیای سرگرمی اتفاقات زیاد و جالبی را با خودش به همراه آورده بود درست مثل سال گذشته امسال هم بهترین های سال را در بخش های مختلفی انتخاب کرده ایم و فیلم هم یکی از این بخش ها است طرفدار هر سبکی که باشید مطمئنا امسال فیلمی خوب برای دیدن پیدا کرده اید از فیلم های ابرقهرمانی خوبی مثل جنگ داخلی گرفته تا انیمیشن هایی مثل زوتوپیا یا موآنا و حتی فیلم های عملی تخیلی بدرد بخور فیلم های امسال کم و بیش به مذاق همه خوش آمده است طبق نظرسنجی بلند و بالایی که بین تحریریه ی دیجی کالامگ و عده ای از فیلم دوستان دور و برمان داشته ایم از بین فیلم امسال به پنج فیلم برتر سال رسیده ایم تاکید می کنیم که چیزی به اسم بهترین مطلق وجود ندارد و این فیلم ها پس از رای گیری بین بر و بچه های دیجی کالامگ به دست آمده اند حتما در بخش نظرات از بهترین های فیلم هایی که امسال دیده اید برای ما و دیگران بگویید اگر شاهکاری به اسم محافظان کهکشان را کنار بگذاریم در بین فیلم های ابرقهرمانی چند سال اخیر فکر کنیم به سادگی بتوانیم بگوییم که دکتر استرنج یک سر و گردن بال

In [24]:
class digimagTestDataset(torch.utils.data.Dataset):

    def __init__(self, tokenizer, text, targets=None, label_list=None, max_len=128):
        self.text = text
        self.targets = targets
        self.has_target = isinstance(targets, list) or isinstance(targets, np.ndarray)

        self.tokenizer = tokenizer
        self.max_len = max_len


        self.label_map = {
                      "عمومی" :0,
                      "بقیه" :6,
                      "سلامت و زیبایی" : 2,
                      "راهنمای خرید" : 3,
                      "کتاب و ادبیات" : 5,
                  }

    def __len__(self):
        return len(self.text)

    def __getitem__(self, item):
        text = str(self.text[item])


        if self.has_target:
            # Ensure target is an integer
            target = self.label_map.get(str(self.targets[item]), self.targets[item])
            target = int(target)

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')

        inputs = {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

        if self.has_target:
            inputs['targets'] = torch.tensor(target, dtype=torch.long)

        return inputs


def create_data_loader(x, y, tokenizer, max_len, batch_size, label_list):
    dataset = digimagTestDataset(
        text=x,
        targets=y,
        tokenizer=tokenizer,
        max_len=max_len,
        label_list=label_list)

    return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [25]:
label_list = labels
train_data_loader = create_data_loader(train_df['text'].to_numpy(), train_df['label'].to_numpy(), tokenizer, MAX_LEN, TRAIN_BATCH_SIZE, label_list)
valid_data_loader = create_data_loader(dev_df['text'].to_numpy(), dev_df['label'].to_numpy(), tokenizer, MAX_LEN, VALID_BATCH_SIZE, label_list)
test_data_loader = create_data_loader(test_df['text'].to_numpy(), None, tokenizer, MAX_LEN, TEST_BATCH_SIZE, label_list)

In [26]:
train_df

,text,label
0,نمایش تبلیغ در لاک اسکرین تعدادی از گوشی های ه...,3
1,شکست در باکس آفیس پس از بازخوردهای بد طرفداران...,5
2,کلاسیک بینی همه چیز در یک شب اتفاق افتاد فیلم ...,5
3,اپل دوباره سراغ رنده رفته چراکه آپگرید کردن سط...,3
4,بررسی جز به جز بهترین بخش جعبه ابزار بازی سازا...,0
...,...,...
6891,چند ساعت خواب برای بدن شما مفید است شایع است ک...,2
6892,فیلم مگامن ساخته می شود شرکت بازی سازی کپ کام ...,0
6893,تعداد حملات بدافزاری به سیستم عامل مکینتاش برا...,3
6894,چطور از مبتلا شدن به آنفولانزا پیشگیری کنیم فر...,2


In [27]:
sample_data = next(iter(train_data_loader))

print(sample_data.keys())

print(sample_data['text'])
print(sample_data['input_ids'].shape)
print(sample_data['input_ids'][0, :])
print(sample_data['attention_mask'].shape)
print(sample_data['attention_mask'][0, :])
print(sample_data['token_type_ids'].shape)
print(sample_data['token_type_ids'][0, :])
print(sample_data['targets'].shape)
print(sample_data['targets'][0])

dict_keys(['text', 'input_ids', 'attention_mask', 'token_type_ids', 'targets'])
['نمایش تبلیغ در لاک اسکرین تعدادی از گوشی های هواوی روز گذشته تعداد زیادی از کاربران گوشی های هواوی در توییتر اعلام کردند که به صورت ناگهانی تبلیغی در لاک اسکرین گوشی هایشان پدیدار شده است وجه مشترک این کاربران هم نمایش تبلیغ سایت است به گزارش ظاهرا این تبلیغ در تعدادی از گوشی ها به نمایش درآمده که از قابلیت بهره می برند این قابلیت به کاربران گوشی های هواوی اجازه می دهد که با هر بار آنلاک کردن گوشی در لاک اسکرین خود شاهد تصویر جدیدی باشند این تبلیغ فقط مختص به کشور خاصی نیست و کاربران کشورهایی مانند بریتانیا هلند ایرلند آفریقای جنوبی و آلمان هم با این پدیده مواجه شده اند اکانت توییتر هواوی در آلمان در پاسخ به یکی از کاربران نوشته از این بابت که فکر می کنید ما قصد نمایش تبلیغات داریم متاسفیم کاربران می گویند با مراجعه به بخش تنظیمات و تغییر تصویر پس زمینه به مواردی که پیش فرض گوشی نیستند می توانند از دست این تبلیغ خلاص شوند هواوی می گوید که در این زمینه آن ها مقصر نیستند و این تبلیغات ناشی از اپلیکیشن یا سر

In [43]:
sample_test = next(iter(test_data_loader))
print(sample_test.keys())

dict_keys(['text', 'input_ids', 'attention_mask', 'token_type_ids'])


In [28]:
import torch, gc

gc.collect()
torch.cuda.empty_cache()

!nvidia-smi

Mon May 20 16:01:08 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [39]:
class SentimentModel(nn.Module):

    def __init__(self, config):
        super(SentimentModel, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_NAME_OR_PATH)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids):
        _, pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [40]:
pt_model = SentimentModel(config=config)
pt_model = pt_model.to(device)

print('pt_model', type(pt_model))

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertModel were not initialized from the model checkpoint at ViraIntelligentDataMining/AriaBERT and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.laye

pt_model <class '__main__.SentimentModel'>


In [44]:
# sample data output

sample_data_comment = sample_data['text']
sample_data_input_ids = sample_data['input_ids']
sample_data_attention_mask = sample_data['attention_mask']
sample_data_token_type_ids = sample_data['token_type_ids']
sample_data_targets = sample_data['targets']

# available for using in GPU
sample_data_input_ids = sample_data_input_ids.to(device)
sample_data_attention_mask = sample_data_attention_mask.to(device)
sample_data_token_type_ids = sample_data_token_type_ids.to(device)
sample_data_targets = sample_data_targets.to(device)


# outputs = F.softmax(
#     pt_model(sample_data_input_ids, sample_data_attention_mask, sample_data_token_type_ids),
#     dim=1)

outputs = pt_model(sample_data_input_ids, sample_data_attention_mask, sample_data_token_type_ids)
_, preds = torch.max(outputs, dim=1)

print(outputs[:5, :])
print(preds[:5])

TypeError: dropout(): argument 'input' (position 1) must be Tensor, not str

In [45]:
sample_data_input_ids

tensor([[    0, 12919,  2087,  ...,   317,  2651,     2],
        [    0, 19109,   297,  ...,  1049,  4173,     2],
        [    0, 51115,  2754,  ...,   993, 10687,     2],
        ...,
        [    0, 46296,   337,  ...,  1403,   770,     2],
        [    0,  4340,   537,  ..., 31816,  4577,     2],
        [    0,  2211,  2996,  ...,   595,  1676,     2]], device='cuda:0')

In [41]:
def simple_accuracy(y_true, y_pred):
    return (y_true == y_pred).mean()

def acc_and_f1(y_true, y_pred, average='weighted'):
    acc = simple_accuracy(y_true, y_pred)
    f1 = f1_score(y_true=y_true, y_pred=y_pred, average=average)
    return {
        "acc": acc,
        "f1": f1,
    }

def y_loss(y_true, y_pred, losses):
    y_true = torch.stack(y_true).cpu().detach().numpy()
    y_pred = torch.stack(y_pred).cpu().detach().numpy()
    y = [y_true, y_pred]
    loss = np.mean(losses)

    return y, loss


def eval_op(model, data_loader, loss_fn):
    model.eval()

    losses = []
    y_pred = []
    y_true = []

    with torch.no_grad():
        for dl in tqdm(data_loader, total=len(data_loader), desc="Evaluation... "):

            input_ids = dl['input_ids']
            attention_mask = dl['attention_mask']
            token_type_ids = dl['token_type_ids']
            targets = dl['targets']

            # move tensors to GPU if CUDA is available
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            targets = targets.to(device)

            # compute predicted outputs by passing inputs to the model
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids)

            # convert output probabilities to predicted class
            _, preds = torch.max(outputs, dim=1)

            # calculate the batch loss
            loss = loss_fn(outputs, targets)

            # accumulate all the losses
            losses.append(loss.item())

            y_pred.extend(preds)
            y_true.extend(targets)

    eval_y, eval_loss = y_loss(y_true, y_pred, losses)
    return eval_y, eval_loss


def train_op(model,
             data_loader,
             loss_fn,
             optimizer,
             scheduler,
             step=0,
             print_every_step=100,
             eval=False,
             eval_cb=None,
             eval_loss_min=np.Inf,
             eval_data_loader=None,
             clip=0.0):

    model.train()

    losses = []
    y_pred = []
    y_true = []

    for dl in tqdm(data_loader, total=len(data_loader), desc="Training... "):
        step += 1

        input_ids = dl['input_ids']
        attention_mask = dl['attention_mask']
        token_type_ids = dl['token_type_ids']
        targets = dl['targets']

        # move tensors to GPU if CUDA is available
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        targets = targets.to(device)

        # clear the gradients of all optimized variables
        optimizer.zero_grad()

        # compute predicted outputs by passing inputs to the model
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids)

        # convert output probabilities to predicted class
        _, preds = torch.max(outputs, dim=1)

        # calculate the batch loss
        loss = loss_fn(outputs, targets)

        # accumulate all the losses
        losses.append(loss.item())

        # compute gradient of the loss with respect to model parameters
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        if clip > 0.0:
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)

        # perform optimization step
        optimizer.step()

        # perform scheduler step
        scheduler.step()

        y_pred.extend(preds)
        y_true.extend(targets)

        if eval:
            train_y, train_loss = y_loss(y_true, y_pred, losses)
            train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

            if step % print_every_step == 0:
                eval_y, eval_loss = eval_op(model, eval_data_loader, loss_fn)
                eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

                if hasattr(eval_cb, '__call__'):
                    eval_loss_min = eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min)

    train_y, train_loss = y_loss(y_true, y_pred, losses)

    return train_y, train_loss, step, eval_loss_min

In [42]:
optimizer = AdamW(pt_model.parameters(), lr=LEARNING_RATE, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss()

step = 0
eval_loss_min = np.Inf
history = collections.defaultdict(list)


def eval_callback(epoch, epochs, output_path):
    def eval_cb(model, step, train_score, train_loss, eval_score, eval_loss, eval_loss_min):
        statement = ''
        statement += 'Epoch: {}/{}...'.format(epoch, epochs)
        statement += 'Step: {}...'.format(step)

        statement += 'Train Loss: {:.6f}...'.format(train_loss)
        statement += 'Train Acc: {:.3f}...'.format(train_score['acc'])

        statement += 'Valid Loss: {:.6f}...'.format(eval_loss)
        statement += 'Valid Acc: {:.3f}...'.format(eval_score['acc'])

        print(statement)

        if eval_loss <= eval_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                eval_loss_min,
                eval_loss))

            torch.save(model.state_dict(), output_path)
            eval_loss_min = eval_loss

        return eval_loss_min


    return eval_cb


for epoch in tqdm(range(1, EPOCHS + 1), desc="Epochs... "):
    train_y, train_loss, step, eval_loss_min = train_op(
        model=pt_model,
        data_loader=train_data_loader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        scheduler=scheduler,
        step=step,
        print_every_step=EEVERY_EPOCH,
        eval=True,
        eval_cb=eval_callback(epoch, EPOCHS, OUTPUT_PATH),
        eval_loss_min=eval_loss_min,
        eval_data_loader=valid_data_loader,
        clip=CLIP)

    train_score = acc_and_f1(train_y[0], train_y[1], average='weighted')

    eval_y, eval_loss = eval_op(
        model=pt_model,
        data_loader=valid_data_loader,
        loss_fn=loss_fn)

    eval_score = acc_and_f1(eval_y[0], eval_y[1], average='weighted')

    history['train_acc'].append(train_score['acc'])
    history['train_loss'].append(train_loss)
    history['val_acc'].append(eval_score['acc'])
    history['val_loss'].append(eval_loss)

Epochs... :   0%|          | 0/3 [00:00<?, ?it/s]

Training... :   0%|          | 0/431 [00:00<?, ?it/s]

TypeError: dropout(): argument 'input' (position 1) must be Tensor, not str